In [1]:
# This notebook supports input parameters for automatic report generation. The parameters must be variables in this
# cell, which has a special 'parameters' tag.
DATA_ROOT = r'G:\Shared drives\Covid-19 Spectrum Monitoring\Data'

HISTOGRAM_RESOLUTION_SWEEPS = 100
HISTOGRAM_POWER_LOW = -120
HISTOGRAM_POWER_HIGH = -20

dat_path_pattern = 'dUsVcuPP/*2020-11-06*.dat'

figure_format = 'svg'

In [2]:
# hide the pink warnings in reports (comment to include them)
from environment import *
import figures
import histogram_analysis
from dwell_analysis import dBtopow, powtodB

bounds = (HISTOGRAM_POWER_LOW, HISTOGRAM_POWER_HIGH)

set_matplotlib_formats(figure_format)

display(widgets.HTML(f'This report was produced {time.strftime(time_format)}'));

def normalize(data,bystd=False):
    if isinstance(data,(pd.Series,pd.DataFrame)):
        v = data.values
    else:
        v = data
    if bystd:
        return (v-np.nanmedan(v))/v.nanmedian(np)
    else:
        return v/np.nanmean(v)

HTML(value='This report was produced 2021-09-14 08:36:19')

In [ ]:
from scipy.signal import chirp, find_peaks, peak_widths, argrelmax

import matplotlib.pyplot as plt

def dwell_occupancy_by_threshold(dwell, prominence_dB=3, rel_height_dB=-6):
    if isinstance(dwell, (pd.Series, pd.DataFrame)):
        dwell = dwell.values
    dwell = np.squeeze(dwell)
    
    ii = np.arange(dwell.size)[:,np.newaxis]
    
    try:
        peaks, _ = find_peaks(powtodB(dwell), prominence=prominence_dB)
    except:
        print(dwell)
        raise

    rel_height = rel_height=1-10**(rel_height_dB/10.)
    widths, widths_heights, ips_left, ips_right = peak_widths(dwell, peaks, rel_height)

    active_inds = (
        (ii >= np.ceil(ips_left) ) & (ii <= np.floor(ips_right))
    ).any(axis=1).astype('bool')
    
    on = dwell[active_inds]
    off = dwell[~active_inds]

    counts, bins = np.histogram(widths, bins=np.arange(40))

    width_info = pd.Series(
        counts,
        index=bins[:-1]
    )

    width_info['Off power'] = powtodB(np.median(off))
    return width_info


# %timeit -n1 -r1 glob.iloc[:2000].apply(dwell_occupancy_by_threshold, axis=1)

In [ ]:
relmaxima = argrelmax(glob.values[:20], axis=1)

In [ ]:

def plot_tagged_dwell_window(dwell, prominence_dB=3, rel_height_dB=-6):
    x = dwell.values
    x = np.squeeze(dwell.values)

    elapsed = (dwell.index.values - dwell.index.values[0])*1000
    
    ii = np.arange(x.size)[:,np.newaxis]

    peaks, _ = find_peaks(powtodB(x), prominence=prominence_dB)

    rel_height = rel_height=1-10**(rel_height_dB/10.)
    widths, widths_heights, ips_left, ips_right = peak_widths(x, peaks, rel_height)

    active_inds = (
        (ii >= np.ceil(ips_left) ) & (ii <= np.floor(ips_right))
    ).any(axis=1).astype('bool')

    on = x[active_inds]
    off = x[~active_inds]

    counts, bins = np.histogram(widths, bins=np.arange(40))

    width_info = pd.Series(
        counts,
        index=bins[:-1]
    )

    width_info['Off power'] = powtodB(np.median(off))
    ips_left = np.array(ips_left).astype(int)
    ips_right = np.ceil(np.array(ips_right)).astype(int)

    figure()
    plot(elapsed, powtodB(x), marker='.', ms=2)
    plot(elapsed[peaks], powtodB(x[peaks]), "x")
    hlines(powtodB(widths_heights), elapsed[ips_left], elapsed[ips_right], color="C2")
    ylabel(figures.power_label)
    xlabel('Dwell window time elapsed (ms)')    
    
    figure()
    plot(elapsed[active_inds], powtodB(on), '.')
    plot(elapsed[~active_inds], powtodB(off), '.')
    legend(
        ['Samples tagged within peaks', 'Tagged outside of peaks'],
        title=(
            f"Site {dwell.name[0]}\n"
            f"{dwell.name[-1]:0.1f} MHz\n"
            f"{str(ts)})"
        )
    )
    ylabel(figures.power_label)
    xlabel('Dwell window time elapsed (ms)')

    print(np.mean(powtodB(on)), np.median(powtodB(off)))

In [ ]:
maxima = glob.loc(axis=0)[:,:,2462.].max(axis=1)
plot_tagged_dwell_window(glob.loc(axis=0)[maxima.idxmax()])
plot_tagged_dwell_window(glob.loc(axis=0)[maxima.idxmin()])

In [ ]:
maxima = glob.loc(axis=0)[:,:,701.5].max(axis=1)
plot_tagged_dwell_window(glob.loc(axis=0)[maxima.idxmax()])
plot_tagged_dwell_window(glob.loc(axis=0)[maxima.idxmin()])

In [ ]:
maxima = glob.loc(axis=0)[:,:,739.].max(axis=1)
plot_tagged_dwell_window(glob.loc(axis=0)[maxima.idxmax()])
plot_tagged_dwell_window(glob.loc(axis=0)[maxima.idxmin()])

In [ ]:
maxima = glob.loc(axis=0)[:,:,2695.].max(axis=1)
plot_tagged_dwell_window(glob.loc(axis=0)[maxima.idxmax()])
plot_tagged_dwell_window(glob.loc(axis=0)[maxima.idxmin()])